In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import joblib

In [3]:
# reading the final dataset
dataset = pd.read_csv('datasets/final_dataset.csv')
dataset

,molecule_chembl_id,standard_value,canonical_smiles,pIC50,mol_Wt,cLogP,H_Acceptors,H_Donors,PubchemFP0,PubchemFP1,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,CHEMBL112021,54.00,Clc1ccccc1Cn1cc(Cn2ccnc2)c2ccccc21,7.267606,321.811,4.58780,3.0,0.0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,CHEMBL113637,57.00,CCn1cc(C(c2ccc(F)cc2)n2ccnc2)c2ccccc21,7.244125,319.383,4.63450,3.0,0.0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,CHEMBL111868,78.50,Cn1cc(C(c2ccc(F)cc2)n2ccnc2)c2cc(Br)ccc21,7.105130,384.252,4.91410,3.0,0.0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,CHEMBL41761,41.00,CCn1ccc2cc(C(c3ccc(F)cc3)n3ccnc3)ccc21,7.387216,319.383,4.63450,3.0,0.0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,CHEMBL431859,238.00,CCn1c(C(c2ccc(F)cc2)n2ccnc2)c(C)c2cc(Br)ccc21,6.623423,412.306,5.70542,3.0,0.0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2110,CHEMBL5203413,0.09,CC#CCOc1ccc2cc(C(c3ccc(C#N)cc3)n3cncn3)oc2c1,10.045757,368.396,3.93578,6.0,0.0,1,1,...,0,0,0,0,0,0,0,0,0,0
2111,CHEMBL5177928,42.00,COc1ccc2cc(/C=N/NC3=NC(=O)CS3)ccc2c1,7.376751,299.355,2.40130,5.0,1.0,1,1,...,0,0,0,0,0,0,0,0,0,0
2112,CHEMBL5179009,0.72,CCC#CCOc1ccc2cc(C(c3ccc(C#N)cc3)n3cncn3)oc2c1,9.142668,382.423,4.32588,6.0,0.0,1,1,...,0,0,0,0,0,0,0,0,0,0
2113,CHEMBL5176279,31.00,CCOC(=O)Cc1csc(N/N=C/c2ccc3cc(OC)ccc3c2)n1,7.508638,369.446,3.85650,7.0,1.0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = dataset.drop(columns=['molecule_chembl_id', 'standard_value', 'canonical_smiles', 'pIC50'])
X

,mol_Wt,cLogP,H_Acceptors,H_Donors,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,321.811,4.58780,3.0,0.0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,319.383,4.63450,3.0,0.0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,384.252,4.91410,3.0,0.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,319.383,4.63450,3.0,0.0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,412.306,5.70542,3.0,0.0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2110,368.396,3.93578,6.0,0.0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2111,299.355,2.40130,5.0,1.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2112,382.423,4.32588,6.0,0.0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2113,369.446,3.85650,7.0,1.0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
!mkdir -p 'datasets/model'
X.to_csv('datasets/model/X.csv', index=False)

In [6]:
y = dataset['pIC50']
y.to_csv('datasets/model/y.csv', index=False)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1692, 885), (423, 885), (1692,), (423,))

### Feature Selection using VarianceThreshold

In [8]:
# drop constant cols only
var_thres = VarianceThreshold(threshold=0)
var_thres.fit(X_train)
constant_cols = [column for column in X_train.columns
                 if column not in X_train.columns[var_thres.get_support()]]
print (len(constant_cols))

# drop constant cols
X_train = X_train.drop(columns=constant_cols, axis=1)
X_test = X_test.drop(columns=constant_cols, axis=1)

X_train.shape, X_test.shape

310


((1692, 575), (423, 575))

### Feature Selection using Pearson Correlation Matrix

In [9]:
correlation_matrix = X_train.corr()
correlation_matrix

,mol_Wt,cLogP,H_Acceptors,H_Donors,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP11,PubchemFP12,...,PubchemFP835,PubchemFP836,PubchemFP839,PubchemFP840,PubchemFP842,PubchemFP844,PubchemFP854,PubchemFP860,PubchemFP861,PubchemFP863
mol_Wt,1.000000,0.506884,0.571857,0.378044,0.050402,0.049061,0.398405,0.432593,0.050402,0.434954,...,0.025268,0.040640,-0.023887,-0.003745,-0.024383,0.004942,-0.020832,-0.052534,-0.003745,-0.033291
cLogP,0.506884,1.000000,-0.238248,-0.013202,0.081829,0.037134,0.350049,0.438576,0.081829,0.445302,...,-0.017526,0.054979,0.048435,0.084639,-0.010910,0.026723,-0.021168,0.237644,0.084639,-0.016707
H_Acceptors,0.571857,-0.238248,1.000000,0.308131,0.024698,0.003298,0.068998,-0.023023,0.024698,0.151438,...,0.053007,0.000970,-0.105688,-0.115715,-0.017643,-0.037327,0.000560,-0.369794,-0.115715,-0.021365
H_Donors,0.378044,-0.013202,0.308131,1.000000,-0.029130,-0.001104,0.143591,0.226495,-0.029130,0.023214,...,-0.019848,-0.001121,-0.021295,0.063780,-0.041175,-0.015535,-0.015535,-0.154676,0.063780,-0.034779
PubchemFP0,0.050402,0.081829,0.024698,-0.029130,1.000000,0.377293,0.029849,0.006318,1.000000,0.044138,...,0.001451,0.001025,0.003636,0.003585,0.001567,0.000591,0.000591,0.011744,0.003585,0.001324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PubchemFP844,0.004942,0.026723,-0.037327,-0.015535,0.000591,0.001567,0.019812,-0.006318,0.000591,0.013398,...,-0.001451,-0.001025,-0.003636,-0.003585,-0.001567,1.000000,-0.000591,0.050354,-0.003585,-0.001324
PubchemFP854,-0.020832,-0.021168,0.000560,-0.015535,0.000591,0.001567,-0.029849,-0.006318,0.000591,-0.044138,...,-0.001451,-0.001025,0.162640,-0.003585,0.377293,-0.000591,1.000000,0.050354,-0.003585,0.446684
PubchemFP860,-0.052534,0.237644,-0.369794,-0.154676,0.011744,0.031128,0.199282,0.221760,0.011744,0.123247,...,-0.028810,-0.020354,0.278644,0.305297,0.133460,0.050354,0.050354,1.000000,0.305297,0.112727
PubchemFP861,-0.003745,0.084639,-0.115715,0.063780,0.003585,0.009503,0.120119,0.197272,0.003585,0.071543,...,-0.008796,-0.006214,0.061966,1.000000,-0.009503,-0.003585,-0.003585,0.305297,1.000000,-0.008027


In [10]:
def correlation(dataset, threshold):
  col_corr = set()
  cor_matrix = dataset.corr()
  for i in range(len(cor_matrix.columns)):
    for j in range(i):
      if abs(cor_matrix.iloc[i, j]) > threshold:
        colname = cor_matrix.columns[i]
        col_corr.add(colname)
  return col_corr

In [11]:
corr_features = correlation(X_train, 0.9)
len(set(corr_features))

251

In [12]:
# drop correlated features
X_train = X_train.drop(columns=corr_features, axis=1)
X_test = X_test.drop(columns=corr_features, axis=1)
X_train.shape, X_test.shape

((1692, 324), (423, 324))

In [13]:
## Save these datasets
X_train.to_csv('datasets/model/training_X.csv', index=False)
X_test.to_csv('datasets/model/test_X.csv', index=False)


In [14]:
## Finding best parameters for RandomForestRegressor
param_rf = {'n_estimators': [10, 20, 30,40, 50, 60],'max_features': [1, 'sqrt', 'log2'], 'min_samples_split': [2,4,8], 'bootstrap':[True, False]}
grid_rf = GridSearchCV(RandomForestRegressor(),param_rf, verbose = 3, cv = 5)
grid_rf.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV 1/5] END bootstrap=True, max_features=1, min_samples_split=2, n_estimators=10;, score=0.524 total time=   0.0s
[CV 2/5] END bootstrap=True, max_features=1, min_samples_split=2, n_estimators=10;, score=0.500 total time=   0.0s
[CV 3/5] END bootstrap=True, max_features=1, min_samples_split=2, n_estimators=10;, score=0.496 total time=   0.0s
[CV 4/5] END bootstrap=True, max_features=1, min_samples_split=2, n_estimators=10;, score=0.398 total time=   0.0s
[CV 5/5] END bootstrap=True, max_features=1, min_samples_split=2, n_estimators=10;, score=0.540 total time=   0.0s
[CV 1/5] END bootstrap=True, max_features=1, min_samples_split=2, n_estimators=20;, score=0.509 total time=   0.0s
[CV 2/5] END bootstrap=True, max_features=1, min_samples_split=2, n_estimators=20;, score=0.537 total time=   0.0s
[CV 3/5] END bootstrap=True, max_features=1, min_samples_split=2, n_estimators=20;, score=0.524 total time=   0.0s
[CV 4/5] END boot

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'bootstrap': [True, False],
                         'max_features': [1, 'sqrt', 'log2'],
                         'min_samples_split': [2, 4, 8],
                         'n_estimators': [10, 20, 30, 40, 50, 60]},
             verbose=3)

In [15]:
grid_rf.best_params_

{'bootstrap': False,
 'max_features': 'sqrt',
 'min_samples_split': 8,
 'n_estimators': 50}

In [16]:
grid_rf.best_estimator_

RandomForestRegressor(bootstrap=False, max_features='sqrt', min_samples_split=8,
                      n_estimators=50)

In [17]:
# build a model using these parameters
rf_regressor = RandomForestRegressor(n_estimators=50,bootstrap = False, max_features = 'sqrt', min_samples_split=8)
rf_regressor.fit(X_train, y_train)
y_pred = rf_regressor.predict(X_test)

In [18]:
print('Mean Squared Error :', mean_squared_error(y_test, y_pred))
print('R2 Score : ', r2_score(y_test, y_pred))

Mean Squared Error : 0.7236927521264828
R2 Score :  0.5809425846718457


In [19]:
!mkdir -p 'model'
joblib.dump(rf_regressor, 'model/rf_model.joblib')

['model/rf_model.joblib']

In [20]:
## Finding best parameters for SVR
param_SVR = {'kernel':['rbf', 'linear', 'sigmoid'],
             'gamma': ['scale', 'auto'],
             'C': [0.001, 0.01, 0.1, 1, 10, 20, 50]}
grid = GridSearchCV(SVR(),param_SVR, verbose = 3, cv=5)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits
[CV 1/5] END .C=0.001, gamma=scale, kernel=rbf;, score=-0.040 total time=   0.4s
[CV 2/5] END .C=0.001, gamma=scale, kernel=rbf;, score=-0.010 total time=   0.4s
[CV 3/5] END .C=0.001, gamma=scale, kernel=rbf;, score=-0.001 total time=   0.4s
[CV 4/5] END .C=0.001, gamma=scale, kernel=rbf;, score=-0.018 total time=   0.4s
[CV 5/5] END .C=0.001, gamma=scale, kernel=rbf;, score=-0.017 total time=   0.4s
[CV 1/5] END C=0.001, gamma=scale, kernel=linear;, score=0.242 total time=   0.3s
[CV 2/5] END C=0.001, gamma=scale, kernel=linear;, score=0.214 total time=   0.4s
[CV 3/5] END C=0.001, gamma=scale, kernel=linear;, score=0.209 total time=   0.4s
[CV 4/5] END C=0.001, gamma=scale, kernel=linear;, score=0.218 total time=   0.4s
[CV 5/5] END C=0.001, gamma=scale, kernel=linear;, score=0.281 total time=   0.4s
[CV 1/5] END C=0.001, gamma=scale, kernel=sigmoid;, score=-0.041 total time=   0.3s
[CV 2/5] END C=0.001, gamma=scale, kern